In [68]:
import sqlite3
import time
import tracemalloc
import os

In [92]:
def query_time_comparison(aggregation, table):
    conn = sqlite3.connect('../../../data/tpch.db')
    cursor = conn.cursor()
    
    start = time.time()
    tracemalloc.start()
    process = psutil.Process(os.getpid())
    io_start = process.io_counters()
    
    cursor.execute(f"SELECT {aggregation} FROM {table}")
    output = cursor.fetchone()[0]
    
    io_end = process.io_counters()
    curr_mem, max_mem = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    stop = time.time()
    diff = stop - start
    
    bytes_read = io_end.read_bytes - io_start.read_bytes
    
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    size = cursor.fetchone()[0]
    return diff, output, size, max_mem, bytes_read

In [101]:
def query_stats(queries):
    for i in queries:
        exec_time, result, table_size, memory, read = query_time_comparison(i[0], i[1])
        print(f"Query: {i[0]}, Exec Time: {exec_time * 1000} ms, {table_size} rows, using {memory / 10 ** 6} MB, read {read} bytes")

In [102]:
uni_vs_semi_norm = (("1", "CUSTOMER"), #just a dummy to clear out malloc / bytes clutter
                    ("MAX(L_QUANTITY)", "LINEITEM"), ("MAX(L_EXTENDEDPRICE)", "LINEITEM"), 
                    ("MIN(L_QUANTITY)", "LINEITEM"), ("MIN(L_EXTENDEDPRICE)", "LINEITEM"),
                    ("AVG(L_QUANTITY)", "LINEITEM"), ("AVG(L_EXTENDEDPRICE)", "LINEITEM"),
                    ("MAX(P_RETAILPRICE)", "PART"), ("MAX(O_TOTALPRICE)", "ORDERS"),
                    ("MIN(P_RETAILPRICE)", "PART"), ("MIN(O_TOTALPRICE)", "ORDERS"),
                    ("AVG(P_RETAILPRICE)", "PART"), ("AVG(O_TOTALPRICE)", "ORDERS"))

query_stats(uni_vs_semi_norm)

Query: 1, Exec Time: 0.9996891021728516 ms, 150000 rows, using 0.001291 MB, read 16400 bytes
Query: MAX(L_QUANTITY), Exec Time: 1404.4408798217773 ms, 6001215 rows, using 0.150223 MB, read 799535120 bytes
Query: MAX(L_EXTENDEDPRICE), Exec Time: 1496.2201118469238 ms, 6001215 rows, using 0.150247 MB, read 799535120 bytes
Query: MIN(L_QUANTITY), Exec Time: 1384.523868560791 ms, 6001215 rows, using 0.150247 MB, read 799535120 bytes
Query: MIN(L_EXTENDEDPRICE), Exec Time: 1474.0173816680908 ms, 6001215 rows, using 0.150246 MB, read 799535120 bytes
Query: AVG(L_QUANTITY), Exec Time: 1274.0275859832764 ms, 6001215 rows, using 0.150247 MB, read 799535120 bytes
Query: AVG(L_EXTENDEDPRICE), Exec Time: 1308.032512664795 ms, 6001215 rows, using 0.150247 MB, read 799535120 bytes
Query: MAX(P_RETAILPRICE), Exec Time: 48.00081253051758 ms, 200000 rows, using 0.001371 MB, read 24764432 bytes
Query: MAX(O_TOTALPRICE), Exec Time: 328.07207107543945 ms, 1500000 rows, using 0.150359 MB, read 169259024 by

In [103]:
low_vs_high_count = (("1", "CUSTOMER"),
                     ("MAX(P_RETAILPRICE)", "PART"), ("MAX(L_QUANTITY)", "LINEITEM"),
                     ("MIN(P_RETAILPRICE)", "PART"), ("MIN(L_QUANTITY)", "LINEITEM"),
                     ("AVG(P_RETAILPRICE)", "PART"), ("AVG(L_QUANTITY)", "LINEITEM"))

query_stats(low_vs_high_count)

Query: 1, Exec Time: 0.3254413604736328 ms, 150000 rows, using 0.001291 MB, read 16400 bytes
Query: MAX(P_RETAILPRICE), Exec Time: 49.00002479553223 ms, 200000 rows, using 0.001371 MB, read 24764432 bytes
Query: MAX(L_QUANTITY), Exec Time: 1383.2581043243408 ms, 6001215 rows, using 0.150336 MB, read 799535120 bytes
Query: MIN(P_RETAILPRICE), Exec Time: 49.00026321411133 ms, 200000 rows, using 0.001399 MB, read 24764432 bytes
Query: MIN(L_QUANTITY), Exec Time: 1379.3623447418213 ms, 6001215 rows, using 0.150355 MB, read 799535120 bytes
Query: AVG(P_RETAILPRICE), Exec Time: 43.01285743713379 ms, 200000 rows, using 0.001371 MB, read 24764432 bytes
Query: AVG(L_QUANTITY), Exec Time: 1277.653455734253 ms, 6001215 rows, using 0.150355 MB, read 799535120 bytes


In [105]:
low_vs_high_cardinality = (("1", "CUSTOMER"),
                           ("MAX(PS_SUPPLYCOST)", "PARTSUPP"), ("MAX(L_QUANTITY)", "LINEITEM"), ("MAX(S_ACCTBAL)", "SUPPLIER"),
                           ("MIN(PS_SUPPLYCOST)", "PARTSUPP"), ("MIN(L_QUANTITY)", "LINEITEM"), ("MIN(S_ACCTBAL)", "SUPPLIER"),
                           ("AVG(PS_SUPPLYCOST)", "PARTSUPP"), ("AVG(L_QUANTITY)", "LINEITEM"), ("AVG(S_ACCTBAL)", "SUPPLIER"))

query_stats(low_vs_high_cardinality)

Query: 1, Exec Time: 0.0 ms, 150000 rows, using 0.001291 MB, read 16400 bytes
Query: MAX(PS_SUPPLYCOST), Exec Time: 322.3903179168701 ms, 800000 rows, using 0.150235 MB, read 124567568 bytes
Query: MAX(L_QUANTITY), Exec Time: 1392.5058841705322 ms, 6001215 rows, using 0.150243 MB, read 799535120 bytes
Query: MAX(S_ACCTBAL), Exec Time: 5.001068115234375 ms, 10000 rows, using 0.001367 MB, read 1474576 bytes
Query: MIN(PS_SUPPLYCOST), Exec Time: 202.50892639160156 ms, 800000 rows, using 0.001375 MB, read 124567568 bytes
Query: MIN(L_QUANTITY), Exec Time: 1403.273344039917 ms, 6001215 rows, using 0.150244 MB, read 799535120 bytes
Query: MIN(S_ACCTBAL), Exec Time: 4.000425338745117 ms, 10000 rows, using 0.001367 MB, read 1474576 bytes
Query: AVG(PS_SUPPLYCOST), Exec Time: 186.00010871887207 ms, 800000 rows, using 0.001375 MB, read 124567568 bytes
Query: AVG(L_QUANTITY), Exec Time: 1304.9981594085693 ms, 6001215 rows, using 0.150463 MB, read 799535120 bytes
Query: AVG(S_ACCTBAL), Exec Time: 